In [21]:
from dotenv import load_dotenv

load_dotenv()

True

## Local MCP server

In [22]:
from langchain_mcp_adapters.client import MultiServerMCPClient

client = MultiServerMCPClient(
    {
        "local_server": {
                "transport": "stdio",
                "command": "python",
                "args": ["resources/2.1_mcp_server.py"],
            }
    }
)

In [23]:
# get tools
tools = await client.get_tools()

# get resources
resources = await client.get_resources("local_server")
for blob in resources:
    print("URI:", blob.metadata.get("uri"))
    print(blob.as_string()[:500])


# get prompts
prompt = await client.get_prompt("local_server", "prompt")
prompt = prompt[0].content

URI: github://langchain-ai/langchain-mcp-adapters/blob/main/README.md
# LangChain MCP Adapters

This library provides a lightweight wrapper that makes [Anthropic Model Context Protocol (MCP)](https://modelcontextprotocol.io/introduction) tools compatible with [LangChain](https://github.com/langchain-ai/langchain) and [LangGraph](https://github.com/langchain-ai/langgraph).

![MCP](static/img/mcp.png)

> [!note]
> A JavaScript/TypeScript version of this library is also available at [langchainjs](https://github.com/langchain-ai/langchainjs/tree/main/libs/langchain-mc


In [24]:
from langchain.agents import create_agent

agent = create_agent(
    model="gpt-5-nano",
    tools=tools,
    system_prompt=prompt
)

In [25]:
from langchain.messages import HumanMessage

config = {"configurable": {"thread_id": "1"}}

response = await agent.ainvoke(
    {"messages": [HumanMessage(content="Tell me about the langchain-mcp-adapters library")]},
    config=config
)

In [30]:
from pprint import pprint

#pprint(response)
print(response["messages"][-1].content)


Here’s a concise overview of the LangChain MCP Adapters library and how it fits into LangChain, LangGraph, and LangSmith.

What is it
- The LangChain MCP Adapters library provides a bridge between the Anthropic Model Context Protocol (MCP) tools ecosystem and LangChain/LangGraph.
- It converts MCP tools into LangChain- and LangGraph-compatible tools, enabling agents to call and orchestrate MCP tools as if they were native LangChain tools.
- It supports tools across multiple MCP servers, so an agent can pull and use tools from many servers at once.
- It integrates the hundreds of MCP tool servers already published into LangGraph agents, simplifying multi-server tool integration.

Key features and what’s new (based on recent releases)
- Multiserver support: Pull tools from multiple MCP servers and use them together in a single agent.
- Multimodal tool support: Tools can accept/produce images, text, and other modalities.
- Elicitation and clarifications: Easy support for prompt-driven cla

In [27]:
from langchain_core.messages import ToolMessage, AIMessage

for i, m in enumerate(response["messages"]):
    print(i, type(m).__name__)
    if isinstance(m, AIMessage) and getattr(m, "tool_calls", None):
        print("  tool_calls:", m.tool_calls)
    if isinstance(m, ToolMessage):
        print("  tool output:", m.content[:500])


0 HumanMessage
1 AIMessage
  tool_calls: [{'name': 'search_web', 'args': {'query': 'langchain-mcp-adapters'}, 'id': 'call_Q1CZJbgK9zChO8lo6UATYqUs', 'type': 'tool_call'}]
2 ToolMessage
  tool output: [{'type': 'text', 'text': '{\n  "query": "langchain-mcp-adapters",\n  "follow_up_questions": null,\n  "answer": null,\n  "images": [],\n  "results": [\n    {\n      "url": "https://changelog.langchain.com/announcements/mcp-adapters-for-langchain-and-langgraph",\n      "title": "MCP Adapters for LangChain and LangGraph",\n      "content": "# LangChain Changelog. Sign up for our newsletter to stay up to date. # MCP Adapters for LangChain and LangGraph. The **LangChain MCP Adapters** is a package that makes it easy to use **Anthropic Model Context Protocol (MCP) tools** with LangChain & LangGraph. * Converts **MCP tools** into **LangChain- & LangGraph-compatible tools**. * Enables interaction with tools across multiple **MCP servers**. * Seamlessly integrates the **hundreds of tool servers** 

In [28]:
import json
from pprint import pprint
from langchain_core.messages import ToolMessage

def pretty_tool_messages(messages, max_chars=800):
    for i, m in enumerate(messages):
        if not isinstance(m, ToolMessage):
            continue

        print(f"\n--- ToolMessage @ index {i} ---")
        print(f"tool_call_id: {getattr(m, 'tool_call_id', None)}")

        content = m.content

        # Tool outputs are often: list[{"type":"text","text":"{...json...}", ...}]
        if isinstance(content, list):
            for j, item in enumerate(content):
                print(f"\n  [block {j}] type={item.get('type')}")
                txt = item.get("text", "")

                # If the text itself is JSON, parse it
                try:
                    obj = json.loads(txt)
                    # Show the most useful bits first
                    print("  query:", obj.get("query"))
                    results = obj.get("results", [])
                    print(f"  results: {len(results)}")
                    for k, r in enumerate(results[:5], start=1):
                        print(f"    {k}. {r.get('title')}")
                        print(f"       {r.get('url')}")
                        snippet = (r.get("content") or "")[:200].replace("\n", " ")
                        print(f"       snippet: {snippet}...")
                except Exception:
                    # Not JSON; print raw (truncated)
                    print(txt[:max_chars] + ("..." if len(txt) > max_chars else ""))
        else:
            # If it's already a dict/str
            pprint(content)


In [29]:
pretty_tool_messages(response["messages"])



--- ToolMessage @ index 2 ---
tool_call_id: call_Q1CZJbgK9zChO8lo6UATYqUs

  [block 0] type=text
  query: langchain-mcp-adapters
  results: 5
    1. MCP Adapters for LangChain and LangGraph
       https://changelog.langchain.com/announcements/mcp-adapters-for-langchain-and-langgraph
       snippet: # LangChain Changelog. Sign up for our newsletter to stay up to date. # MCP Adapters for LangChain and LangGraph. The **LangChain MCP Adapters** is a package that makes it easy to use **Anthropic Mode...
    2. langchain/mcp-adapters - NPM
       https://www.npmjs.com/package/@langchain/mcp-adapters
       snippet: import{createAgent} from "langchain"; import{ChatOpenAI} from"@langchain/openai"; import{MultiServerMCPClient} from"@langchain/mcp-adapters";// Create client and connect to server const client = new M...
    3. LangChain MCP Adapters 0.2.0
       https://changelog.langchain.com/announcements/langchain-mcp-adapters-0-2-0
       snippet: LangChain - Changelog | LangChain MCP Adapte

## Online MCP

In [35]:
client = MultiServerMCPClient(
    {
        "time": {
            "transport": "stdio",
            "command": "uvx",
            "args": [
                "mcp-server-time",
                "--local-timezone=America/Chicago"
            ]
        }
    }
)

tools = await client.get_tools()

In [36]:
agent = create_agent(
    model="gpt-5-nano",
    tools=tools,
)

In [37]:
question = HumanMessage(content="What time is it?")

response = await agent.ainvoke(
    {"messages": [question]}
)

pprint(response)

{'messages': [HumanMessage(content='What time is it?', additional_kwargs={}, response_metadata={}, id='f475e414-c690-471d-901f-7449a19d8147'),
              AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 218, 'prompt_tokens': 293, 'total_tokens': 511, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 192, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CpbBQ8SMG3rt02Hv1A0PrCDffLPsq', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019b4675-f867-7f32-a5ed-daa4ffcc4184-0', tool_calls=[{'name': 'get_current_time', 'args': {'timezone': 'America/Chicago'}, 'id': 'call_mCdWmlpluemvpJroI4NJyisk', 'type': 'tool_call'}], usage_metadata={'input_tokens': 293, 'output_tokens': 2

In [38]:
print(response["messages"][-1].content)

Current time in Chicago (Central Time): Monday, December 22, 2025, 08:28 AM CST (UTC-6). 

Want this in another timezone? I can convert it for you.
